Copied from [MLFlow Tutorial by Anton T. Ruberts](https://github.com/aruberts/tutorials/blob/main/mlflow/mlflow_experiment_tracking.ipynb)

In [2]:
import mlflow
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
import tensorflow_addons as tfa

from tensorflow.keras.callbacks import EarlyStopping

from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset

import catboost as cb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError
import os

In [4]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 15})

In [9]:
os.environ["MLFLOW_TRACKING_URI"] = "http://192.168.0.97"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://192.168.0.97:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

# We don't really need full-fledged MLflow deployment, we can track locally
# mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Or not configure MLflow at all. Data will then be stored in ./mlruns directory

mlflow.set_experiment("income")

2023/03/04 15:22:04 INFO mlflow.tracking.fluent: Experiment with name 'income' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/11', creation_time=1677936124568, experiment_id='11', last_update_time=1677936124568, lifecycle_stage='active', name='income', tags={}>

# Load Data

In [6]:
dset = fetch_california_housing()
data = dset['data']
y = dset['target']
LABEL = dset['target_names'][0]

NUMERIC_FEATURES = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Longitude', 'Latitude']
FEATURES = NUMERIC_FEATURES

data = pd.DataFrame(data, columns=dset['feature_names'])
data[LABEL] = y

data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [7]:
train_data, test_data = train_test_split(data, test_size=0.2)
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Train dataset shape: (16512, 9)
Test dataset shape: (4128, 9)


## Data Processing

In [8]:
X_train, X_val = train_test_split(train_data, test_size=0.2)

sc = StandardScaler()
X_train.loc[:, NUMERIC_FEATURES] = sc.fit_transform(X_train[NUMERIC_FEATURES])
X_val.loc[:, NUMERIC_FEATURES] = sc.transform(X_val[NUMERIC_FEATURES])
test_data.loc[:, NUMERIC_FEATURES] = sc.transform(test_data[NUMERIC_FEATURES])

# Baseline

We will use **random forest regression** model as our baseline

In [12]:
mlflow.sklearn.autolog(disable=True)

with mlflow.start_run(run_name='rf_baseline'):
    params = {
        "n_estimators": 100,
        "max_depth": 20
    }

    mlflow.set_tag("model_name", "RF")
    mlflow.log_params(params)

    rf = RandomForestRegressor(n_estimators=100, max_depth=20)
    rf.fit(X_train[FEATURES], X_train[LABEL])

    rf_preds = rf.predict(test_data[FEATURES])
    rf_rms = mean_squared_error(test_data[LABEL], rf_preds, squared=False)

    mlflow.log_metric("test_rmse", rf_rms)
    mlflow.sklearn.log_model(rf, "sk_models")

# CatBoost

In [13]:
catb_train_dataset = cb.Pool(X_train[FEATURES], X_train[LABEL])
catb_val_dataset = cb.Pool(X_val[FEATURES], X_val[LABEL])
catb_test_dataset = cb.Pool(test_data[FEATURES], test_data[LABEL])

In [14]:
with mlflow.start_run(run_name="catboost"):
    mlflow.set_tag("model_name", "CatBoost")

    catb = cb.CatBoostRegressor()
    catb.fit(catb_train_dataset, eval_set=catb_val_dataset, early_stopping_rounds=50)
    catb_preds = catb.predict(catb_test_dataset)
    catb_rms = mean_squared_error(test_data[LABEL], catb_preds, squared=False)

    mlflow.log_metric("test_rmse", catb_rms)
    mlflow.catboost.log_model(catb, "cb_models")

Learning rate set to 0.076361
0:	learn: 1.1033848	test: 1.1063460	best: 1.1063460 (0)	total: 61.9ms	remaining: 1m 1s
1:	learn: 1.0583850	test: 1.0606150	best: 1.0606150 (1)	total: 64ms	remaining: 31.9s
2:	learn: 1.0195454	test: 1.0217123	best: 1.0217123 (2)	total: 65.4ms	remaining: 21.7s
3:	learn: 0.9841950	test: 0.9859002	best: 0.9859002 (3)	total: 66.9ms	remaining: 16.7s
4:	learn: 0.9513954	test: 0.9529799	best: 0.9529799 (4)	total: 68.3ms	remaining: 13.6s
5:	learn: 0.9204866	test: 0.9222757	best: 0.9222757 (5)	total: 69.5ms	remaining: 11.5s
6:	learn: 0.8925200	test: 0.8941286	best: 0.8941286 (6)	total: 71.2ms	remaining: 10.1s
7:	learn: 0.8671132	test: 0.8688087	best: 0.8688087 (7)	total: 72.9ms	remaining: 9.04s
8:	learn: 0.8435442	test: 0.8455014	best: 0.8455014 (8)	total: 74.5ms	remaining: 8.21s
9:	learn: 0.8232539	test: 0.8249812	best: 0.8249812 (9)	total: 76.3ms	remaining: 7.56s
10:	learn: 0.8039365	test: 0.8056100	best: 0.8056100 (10)	total: 78.2ms	remaining: 7.03s
11:	learn: 0.

# Multi-Layer Perceptron

In [15]:
def build_mlp(params):
    mlp = Sequential([
        Dense(params["layer1_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer2_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer3_size"], activation=params['activation']),
        Dense(1, activation='relu')
    ])
    return mlp

In [16]:
def train_mlp(mlp, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )
    mlp.compile(
        optimizer=optimizer,
        loss=MeanSquaredError(name="mse"),
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )

    early = EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = mlp.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return mlp

In [17]:
def mlp_mlflow_run(
    name,
    mlp_params,
    train_params,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.log_params(mlp_params)
        mlflow.log_params(train_params)

        mlflow.set_tag("model_name", "MLP")

        mlp = build_mlp(mlp_params)
        mlp = train_mlp(mlp, train_params, train_dataset, val_dataset)
        test_preds = mlp.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds.ravel(), squared=False
        )

        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(mlp, "tf_models")

In [18]:
# To TF Dataset
mlp_train_ds = tf.data.Dataset.from_tensor_slices((X_train[FEATURES], X_train[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_val_ds = tf.data.Dataset.from_tensor_slices((X_val[FEATURES], X_val[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_test_ds = tf.data.Dataset.from_tensor_slices(test_data[FEATURES]).batch(512).prefetch(512)

mlp_params = {
    "layer1_size": 512,
    "layer2_size": 128,
    "layer3_size": 64,
    "dropout_rate": 0.3,
    "activation": 'relu'

}
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

mlp_mlflow_run(
    "mlp_base",
    mlp_params,
    train_params,
    mlp_train_ds,
    mlp_val_ds,
    mlp_test_ds,
    test_data[LABEL],
)

Epoch 1/1000
14/26 [===============>..............] - ETA: 0s - loss: 2.8320 - rmse: 1.6829

2023-03-04 15:25:17.643366: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


26/26 [==============================] - 0s 7ms/step - loss: 2.0034 - rmse: 1.4154 - val_loss: 0.8330 - val_rmse: 0.9127
Epoch 2/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.8491 - rmse: 0.9215 - val_loss: 0.6143 - val_rmse: 0.7838
Epoch 3/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.6381 - rmse: 0.7988 - val_loss: 0.4890 - val_rmse: 0.6993
Epoch 4/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.5340 - rmse: 0.7307 - val_loss: 0.4369 - val_rmse: 0.6609
Epoch 5/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.4812 - rmse: 0.6937 - val_loss: 0.4230 - val_rmse: 0.6504
Epoch 6/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.4598 - rmse: 0.6781 - val_loss: 0.4075 - val_rmse: 0.6383
Epoch 7/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.4444 - rmse: 0.6667 - val_loss: 0.4006 - val_rmse: 0.6329
Epoch 8/1000
26/26 [==============================] - 0s 5ms/step - loss: 0

2023/03/04 15:25:26 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpxi97h6s5/model/data/model/assets


2023/03/04 15:25:30 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpxi97h6s5/model, flavor: tensorflow), fall back to return ['tensorflow==2.11.0']. Set logging level to DEBUG to see the full traceback.


In [19]:
mlp_mlflow_run

<function __main__.mlp_mlflow_run(name, mlp_params, train_params, train_dataset, val_dataset, test_dataset, y_test)>

# Feature Tokenizer + Transformer

In [20]:
# To TF Dataset
train_dataset = df_to_dataset(X_train[FEATURES + [LABEL]], LABEL, shuffle=True)
val_dataset = df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
test_dataset = df_to_dataset(test_data[FEATURES], shuffle=False) # No target, no shuffle

/Users/richardkuodis/miniforge3/envs/mlops/lib/python3.10/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/Users/richardkuodis/miniforge3/envs/mlops/lib/python3.10/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/Users/richardkuodis/miniforge3/envs/mlops/lib/python3.10/site-packages/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.n

In [21]:
def build_fttransformer(
    params_to_log, params_to_skip, out_dim=1, out_activation="relu"
):
    # Define encoder
    ft_encoder = FTTransformerEncoder(
        **params_to_log,
        **params_to_skip
    )
    # Add prediction head to the encoder
    ft_transformer = FTTransformer(
        encoder=ft_encoder,
        out_dim=out_dim,
        out_activation=out_activation,
    )

    return ft_transformer

In [22]:
def train_model(model, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )

    model.compile(
        optimizer=optimizer,
        loss={
            "output": tf.keras.losses.MeanSquaredError(name="mse"),
            "importances": None,
        },
        metrics={
            "output": [tf.keras.metrics.RootMeanSquaredError(name="rmse")],
            "importances": None,
        },
    )

    early = EarlyStopping(
        monitor="val_output_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = model.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return model

In [24]:
def fttransformer_mlflow_run(
    name,
    encoder_params,
    train_params,
    params_to_skip,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.set_tag("model_name", "FTTransformer")

        mlflow.log_params(encoder_params)
        mlflow.log_params(train_params)

        # Build and train
        ft_transformer = build_fttransformer(
            encoder_params,
            params_to_skip,
            out_dim=1,
            out_activation="relu",
        )
        ft_transformer = train_model(
            ft_transformer, train_params, train_dataset, val_dataset
        )
        # Evaluate
        test_preds = ft_transformer.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds["output"].ravel(), squared=False
        )

        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(ft_transformer, "tf_models")

In [26]:
mlflow.tensorflow.autolog(disable=True)

## Linear embeddings

In [25]:
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

params_to_skip = dict(
    numerical_data=X_train[NUMERIC_FEATURES].values,
    categorical_data=None,
    y=X_train[LABEL].values
)

linear_embeddings_params = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type="linear",
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='linear',
    encoder_params=linear_embeddings_params,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 6s 139ms/step - loss: 1.0874 - output_loss: 1.0874 - output_rmse: 1.0428 - val_loss: 0.6385 - val_output_loss: 0.6385 - val_output_rmse: 0.7990
Epoch 2/1000
26/26 [==============================] - 3s 132ms/step - loss: 0.6039 - output_loss: 0.6039 - output_rmse: 0.7771 - val_loss: 0.5273 - val_output_loss: 0.5273 - val_output_rmse: 0.7262
Epoch 3/1000
26/26 [==============================] - 4s 136ms/step - loss: 0.5329 - output_loss: 0.5329 - output_rmse: 0.7300 - val_loss: 0.4690 - val_output_loss: 0.4690 - val_output_rmse: 0.6848
Epoch 4/1000
26/26 [==============================] - 4s 143ms/step - loss: 0.4924 - output_loss: 0.4924 - output_rmse: 0.7017 - val_loss: 0.5243 - val_output_loss: 0.5243 - val_output_rmse: 0.7241
Epoch 5/1000
26/26 [==============================] - 4s 139ms/step - loss: 0.4783 - output_loss: 0.4783 - output_rmse: 0.6916 - val_loss: 0.4342 - val_output_loss: 0.4342 - val_output_rmse: 0.6589
Epoch 6/10

2023/03/04 15:31:24 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpi664u5jc/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpi664u5jc/model/data/model/assets
2023/03/04 15:31:31 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpi664u5jc/model, flavor: tensorflow), fall back to return ['tensorflow==2.11.0']. Set logging level to DEBUG to see the full traceback.


## Periodic

In [27]:
periodic_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='periodic',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='periodic',
    encoder_params=periodic_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 5s 146ms/step - loss: 1.1429 - output_loss: 1.1429 - output_rmse: 1.0691 - val_loss: 0.6120 - val_output_loss: 0.6120 - val_output_rmse: 0.7823
Epoch 2/1000
26/26 [==============================] - 4s 146ms/step - loss: 0.4917 - output_loss: 0.4917 - output_rmse: 0.7012 - val_loss: 0.3966 - val_output_loss: 0.3966 - val_output_rmse: 0.6298
Epoch 3/1000
26/26 [==============================] - 4s 135ms/step - loss: 0.4100 - output_loss: 0.4100 - output_rmse: 0.6403 - val_loss: 0.3691 - val_output_loss: 0.3691 - val_output_rmse: 0.6075
Epoch 4/1000
26/26 [==============================] - 4s 140ms/step - loss: 0.3738 - output_loss: 0.3738 - output_rmse: 0.6114 - val_loss: 0.3821 - val_output_loss: 0.3821 - val_output_rmse: 0.6181
Epoch 5/1000
26/26 [==============================] - 4s 148ms/step - loss: 0.3409 - output_loss: 0.3409 - output_rmse: 0.5839 - val_loss: 0.3256 - val_output_loss: 0.3256 - val_output_rmse: 0.5706
Epoch 6/10

2023/03/04 15:34:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmprnjeehfw/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmprnjeehfw/model/data/model/assets
2023/03/04 15:34:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmprnjeehfw/model, flavor: tensorflow), fall back to return ['tensorflow==2.11.0']. Set logging level to DEBUG to see the full traceback.


## PLE - Quantile Binning

In [28]:
pleq_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

pleq_params_to_skip = params_to_skip.copy()
pleq_params_to_skip['y'] = None

fttransformer_mlflow_run(
    name='ple_quantile',
    encoder_params=pleq_params_to_log,
    train_params=train_params,
    params_to_skip=pleq_params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 84s 1s/step - loss: 1.0816 - output_loss: 1.0816 - output_rmse: 1.0400 - val_loss: 0.4577 - val_output_loss: 0.4577 - val_output_rmse: 0.6765
Epoch 2/1000
26/26 [==============================] - 4s 139ms/step - loss: 0.4368 - output_loss: 0.4368 - output_rmse: 0.6609 - val_loss: 0.3714 - val_output_loss: 0.3714 - val_output_rmse: 0.6094
Epoch 3/1000
26/26 [==============================] - 4s 147ms/step - loss: 0.3761 - output_loss: 0.3761 - output_rmse: 0.6133 - val_loss: 0.3836 - val_output_loss: 0.3836 - val_output_rmse: 0.6194
Epoch 4/1000
26/26 [==============================] - 4s 150ms/step - loss: 0.3324 - output_loss: 0.3324 - output_rmse: 0.5765 - val_loss: 0.2986 - val_output_loss: 0.2986 - val_output_rmse: 0.5464
Epoch 5/1000
26/26 [==============================] - 4s 154ms/step - loss: 0.3153 - output_loss: 0.3153 - output_rmse: 0.5615 - val_loss: 0.2954 - val_output_loss: 0.2954 - val_output_rmse: 0.5435
Epoch 6/1000

2023/03/04 15:39:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpjtdc96hy/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpjtdc96hy/model/data/model/assets
2023/03/04 15:43:20 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmpjtdc96hy/model, flavor: tensorflow), fall back to return ['tensorflow==2.11.0']. Set logging level to DEBUG to see the full traceback.


## PLE - Target Binning

In [29]:
plet_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
    task='regression',
    ple_tree_params = {
        "min_samples_leaf": 20,
    }
)


fttransformer_mlflow_run(
    name='ple_target',
    encoder_params=plet_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 101s 2s/step - loss: 1.2767 - output_loss: 1.2767 - output_rmse: 1.1299 - val_loss: 0.5779 - val_output_loss: 0.5779 - val_output_rmse: 0.7602
Epoch 2/1000
26/26 [==============================] - 3s 127ms/step - loss: 0.4868 - output_loss: 0.4868 - output_rmse: 0.6977 - val_loss: 0.4167 - val_output_loss: 0.4167 - val_output_rmse: 0.6455
Epoch 3/1000
26/26 [==============================] - 3s 132ms/step - loss: 0.3712 - output_loss: 0.3712 - output_rmse: 0.6092 - val_loss: 0.3339 - val_output_loss: 0.3339 - val_output_rmse: 0.5778
Epoch 4/1000
26/26 [==============================] - 4s 142ms/step - loss: 0.3347 - output_loss: 0.3347 - output_rmse: 0.5785 - val_loss: 0.3103 - val_output_loss: 0.3103 - val_output_rmse: 0.5571
Epoch 5/1000
26/26 [==============================] - 4s 164ms/step - loss: 0.3128 - output_loss: 0.3128 - output_rmse: 0.5593 - val_loss: 0.3013 - val_output_loss: 0.3013 - val_output_rmse: 0.5489
Epoch 6/100

2023/03/04 15:48:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmp1q3ky9zi/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmp1q3ky9zi/model/data/model/assets
2023/03/04 15:52:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/6g/xckc77ps3v72htb_97t7k8440000gn/T/tmp1q3ky9zi/model, flavor: tensorflow), fall back to return ['tensorflow==2.11.0']. Set logging level to DEBUG to see the full traceback.


# Use Model

In [99]:
samples = next(iter(mlp_test_ds.take(1)))
samples = tfds.as_numpy(samples)
samples.shape

(512, 8)

In [100]:
model_id = "runs:/03164c579bac4ea59d03f2895a87c7a2/tf_models" # Take it from MLflow
loaded_model = mlflow.tensorflow.load_model(model_id)

In [101]:
type(loaded_model)

keras.engine.sequential.Sequential

In [102]:
loaded_model.input

<KerasTensor: shape=(None, 8) dtype=float64 (created by layer 'dense_input')>

In [103]:
loaded_model.predict(samples)

16/16 [==============================] - 0s 545us/step


array([[2.33213   ],
       [0.95032686],
       [2.63527   ],
       [1.2351942 ],
       [2.1635928 ],
       [1.8110108 ],
       [2.5984156 ],
       [2.265806  ],
       [2.148579  ],
       [3.4304152 ],
       [2.8183153 ],
       [1.6917448 ],
       [4.472533  ],
       [3.5663292 ],
       [1.2654133 ],
       [0.97675836],
       [1.9846814 ],
       [3.4875808 ],
       [1.71105   ],
       [1.397042  ],
       [4.994873  ],
       [3.3112843 ],
       [1.6407719 ],
       [1.2439666 ],
       [1.7656429 ],
       [2.0898569 ],
       [1.7174497 ],
       [2.542476  ],
       [2.02471   ],
       [2.5685303 ],
       [1.3914275 ],
       [1.050379  ],
       [2.3030503 ],
       [1.6431    ],
       [1.0579674 ],
       [1.5271425 ],
       [1.224936  ],
       [1.9628086 ],
       [1.5251784 ],
       [1.4307785 ],
       [2.0104992 ],
       [2.5902915 ],
       [1.1670978 ],
       [2.7921162 ],
       [3.0573103 ],
       [2.2484598 ],
       [0.6840952 ],
       [1.475

### Load as PyFunc Model

In [104]:
loaded_model = mlflow.pyfunc.load_model(model_id)

/Users/richardkuodis/miniforge3/envs/mlops/lib/python3.10/site-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


In [105]:
type(loaded_model)

mlflow.pyfunc.PyFuncModel

In [106]:
input = pd.DataFrame([[1,2,3,4,5,6,7,8]])
preds = loaded_model.predict(input)
preds

1/1 [==============================] - 0s 34ms/step


,0
0,0.13907


In [107]:
FEATURES

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Longitude',
 'Latitude']

In [108]:
preds = loaded_model.predict(samples)
preds.shape

16/16 [==============================] - 0s 471us/step


(512, 1)